In [1]:
from p4utils.utils.helper import load_topo
from p4utils.utils.sswitch_thrift_API import SimpleSwitchThriftAPI
import argparse
import shutil
import os
import json
from pprint import pprint
from collections import defaultdict, Counter

import struct
import crcmod

In [2]:
topo = load_topo("topo/fat-tree-k4.json")

m_IS = 7
m_NS = 10

In [7]:
all_paths = []
flow_set = set()
for src_h in topo.get_hosts():
    dst = 's17'
    if src_h == dst: continue
    if (src_h, dst) in flow_set or (dst, src_h) in flow_set: continue
    flow_set.add((src_h, dst))
    all_paths += topo.get_shortest_paths_between_nodes(src_h, dst)
all_paths = set([p[1:-1] for p in all_paths])

In [8]:
path_lens = Counter([len(p) for p in all_paths])
path_lens

Counter({3: 12, 1: 2})

In [41]:
entry_num = 0
for hop_num, path_num in path_lens.items():
    entry_num += hop_num * path_num
entry_num

512

In [22]:
with open("topo/paths-k4.json", "r") as f:
    path_ctrl = json.load(f)

In [25]:
control_count = 0
for ctrl in path_ctrl.values():
    for c in ctrl['controls']:
        if c != 0:
            control_count += 1
control_count

48

In [27]:
entry_num * m_IS, control_count * m_NS

(4368, 480)

In [29]:
(entry_num * m_IS - control_count * m_NS) / entry_num * m_IS


43.61538461538461